In [15]:
import os
import string
from collections import defaultdict
import re
import numpy as np
import pandas as pd

np.random.seed(42)

In [16]:
def load_data(path: str):
    df = pd.read_csv(path)
    return df.to_numpy()

In [17]:
data = load_data("./dataset/iris.csv")
data

array([[5.1, 3.5, 1.4, 0.2, 'Setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Setosa'],
       [5.4, 3.9, 1.7, 0.4, 'Setosa'],
       [4.6, 3.4, 1.4, 0.3, 'Setosa'],
       [5.0, 3.4, 1.5, 0.2, 'Setosa'],
       [4.4, 2.9, 1.4, 0.2, 'Setosa'],
       [4.9, 3.1, 1.5, 0.1, 'Setosa'],
       [5.4, 3.7, 1.5, 0.2, 'Setosa'],
       [4.8, 3.4, 1.6, 0.2, 'Setosa'],
       [4.8, 3.0, 1.4, 0.1, 'Setosa'],
       [4.3, 3.0, 1.1, 0.1, 'Setosa'],
       [5.8, 4.0, 1.2, 0.2, 'Setosa'],
       [5.7, 4.4, 1.5, 0.4, 'Setosa'],
       [5.4, 3.9, 1.3, 0.4, 'Setosa'],
       [5.1, 3.5, 1.4, 0.3, 'Setosa'],
       [5.7, 3.8, 1.7, 0.3, 'Setosa'],
       [5.1, 3.8, 1.5, 0.3, 'Setosa'],
       [5.4, 3.4, 1.7, 0.2, 'Setosa'],
       [5.1, 3.7, 1.5, 0.4, 'Setosa'],
       [4.6, 3.6, 1.0, 0.2, 'Setosa'],
       [5.1, 3.3, 1.7, 0.5, 'Setosa'],
       [4.8, 3.4, 1.9, 0.2, 'Setosa'],
       [5.0, 3.0, 1.6, 0.

In [18]:
def train_test_split(data, train_split_ratio):
    np.random.shuffle(data)
    train_count = int(data.shape[0] * train_split_ratio)
    train = data[:train_count]
    test = data[train_count:]
    X_train = train[:, :4].astype(float)
    y_train = train[:, 4]
    X_test = test[:, :4].astype(float)
    y_test = test[:, 4]
    return X_train, y_train, X_test, y_test

In [19]:
# def encode_labels(labels):
#     return np.where(labels=="setosa", 1, -1)

In [20]:
def binary_cross_entropy(y_true, y_pred):
    m = y_true.shape[0]
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    # Calculating loss
    loss = -1/m * (np.dot(y_true.T, np.log(y_pred)) + np.dot((1 - y_true).T, np.log(1 - y_pred)))

    return loss

# def binary_cross_entropy(y_true, y_pred):
#     return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [21]:
X_train, y_train, X_test, y_test = train_test_split(data, train_split_ratio=0.9)
X_train, y_train, X_test, y_test

(array([[6.1, 2.8, 4.7, 1.2],
        [5.7, 3.8, 1.7, 0.3],
        [7.7, 2.6, 6.9, 2.3],
        [6. , 2.9, 4.5, 1.5],
        [6.8, 2.8, 4.8, 1.4],
        [5.4, 3.4, 1.5, 0.4],
        [5.6, 2.9, 3.6, 1.3],
        [6.9, 3.1, 5.1, 2.3],
        [6.2, 2.2, 4.5, 1.5],
        [5.8, 2.7, 3.9, 1.2],
        [6.5, 3.2, 5.1, 2. ],
        [4.8, 3. , 1.4, 0.1],
        [5.5, 3.5, 1.3, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.1, 3.8, 1.5, 0.3],
        [6.3, 3.3, 4.7, 1.6],
        [6.5, 3. , 5.8, 2.2],
        [5.6, 2.5, 3.9, 1.1],
        [5.7, 2.8, 4.5, 1.3],
        [6.4, 2.8, 5.6, 2.2],
        [4.7, 3.2, 1.6, 0.2],
        [6.1, 3. , 4.9, 1.8],
        [5. , 3.4, 1.6, 0.4],
        [6.4, 2.8, 5.6, 2.1],
        [7.9, 3.8, 6.4, 2. ],
        [6.7, 3. , 5.2, 2.3],
        [6.7, 2.5, 5.8, 1.8],
        [6.8, 3.2, 5.9, 2.3],
        [4.8, 3. , 1.4, 0.3],
        [4.8, 3.1, 1.6, 0.2],
        [4.6, 3.6, 1. , 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [6.7, 3.1, 4.4, 1.4],
        [4

In [22]:
def train(data, labels, epochs, learning_rate):
    num_features = len(data[0])
    weights = np.random.uniform(-0.5, 0.5, num_features)
    bias = np.random.uniform(-0.5, 0.5)

    def activation(x):
        return 1 if x >= 0 else -1

    for epoch in range(epochs):
        ok = 0
        y_pred = []
        for x, y in zip(data, labels):
            output = np.sum(weights * x) + bias
            prediction = activation(output)

            y_pred.append(prediction)
            if prediction != y:
                weights += x * y + learning_rate
                bias += learning_rate * y
            else:
                ok += 1
        # print(labels + 1, np.array(y_pred) + 1)
        loss = binary_cross_entropy(labels + 1, np.array(y_pred) + 1)
        print(f"Epoch {epoch+1}/{epochs} accuracy: {ok/len(data)} loss: {loss}")
    
    return weights, bias

In [23]:
models = {}
for name in ["Setosa", "Virginica", "Versicolor"]:
    y_train_bin = np.where(y_train==name, 1, -1)
    weights, bias = train(X_train, y_train_bin, epochs=15, learning_rate=0.1)
    models[name] = (weights, bias)

Epoch 1/15 accuracy: 0.9481481481481482 loss: -3.9399788126163284
Epoch 2/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 3/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 4/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 5/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 6/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 7/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 8/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 9/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 10/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 11/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 12/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 13/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 14/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 15/15 accuracy: 1.0 loss: -5.730878318305656
Epoch 1/15 accuracy: 0.5851851851851851 loss: 7.521778023831239
Epoch 2/15 accuracy: 0.7703703703703704 loss: 1.910292905992721
Epoch 3/15 accuracy: 0.8148148148148148 loss: 0.47757330143969984
Epoch 4/15 accuracy: 0.84444444444

In [24]:
models

{'Setosa': (array([ 1.44929223,  5.21038292, -8.04444886, -3.77120183]),
  np.float64(-0.32302009017120703)),
 'Virginica': (array([-44.51024855, -29.63877871,  63.82969765,  79.70812038]),
  np.float64(-3.166596243489578)),
 'Versicolor': (array([ -6.72853941, -12.89632792,   3.18657006, -15.007441  ]),
  np.float64(0.6393422419156507))}

In [35]:
def test(data, labels, models):
    codes = {
        "Setosa": [1, -1, -1],
        "Virginica": [-1, 1, -1],
        "Versicolor": [-1, -1, 1]
    }

    def activation(x):
        return 1 if x >= 0 else -1
    
    ok = 0
    for x, y in zip(data, labels):
        pred = []
        for weights, bias in models.values():
            output = np.sum(weights * x) + bias
            prediction = activation(output)
            pred.append(prediction)
        if pred == codes[y]:
            ok += 1
        print(pred, y, codes[y])

    print(f"Accuracy test: {ok/len(data)}")

In [36]:
test(X_test, y_test, models)

[-1, 1, -1] Versicolor [-1, -1, 1]
[-1, 1, -1] Virginica [-1, 1, -1]
[-1, 1, -1] Virginica [-1, 1, -1]
[-1, 1, -1] Versicolor [-1, -1, 1]
[-1, 1, -1] Virginica [-1, 1, -1]
[-1, 1, -1] Versicolor [-1, -1, 1]
[-1, 1, -1] Versicolor [-1, -1, 1]
[-1, 1, -1] Virginica [-1, 1, -1]
[-1, 1, -1] Virginica [-1, 1, -1]
[1, -1, -1] Setosa [1, -1, -1]
[-1, 1, -1] Versicolor [-1, -1, 1]
[-1, 1, -1] Virginica [-1, 1, -1]
[1, -1, -1] Setosa [1, -1, -1]
[-1, 1, -1] Versicolor [-1, -1, 1]
[-1, 1, -1] Virginica [-1, 1, -1]
Accuracy test: 0.6


In [26]:
data, labels = load_data("./dataset/test")
data

PermissionError: [Errno 13] Permission denied: './dataset/test'

In [ ]:
labels = encode_labels(labels)
test(data, labels, weights, bias)

Accuracy test: 0.75


In [ ]:
# def test(weights, bias, text):
#     def activation(x):
#         return 1 if x >= 0 else -1

#     data = letter_freq(text)
#     output = np.sum(weights * data) + bias
#     prediction = activation(output)
#     return "en" if prediction == 1 else "it"

In [ ]:
# text = "SMS Friedrich Carl was an armored cruiser of the Imperial German Navy. A member of the Prinz Adalbert class, the ship was intended to act as a scout for the fleet's battleships and to patrol the German colonial empire. The Prinz Adalbert class was based on the earlier armored cruiser Prinz Heinrich, but with improved armament and armor. Built in the early 1900s, Friedrich Carl served in the German fleet from 1904 to 1909, which included a period as flagship of the reconnaissance squadron and a cruise to the Mediterranean Sea."
# test(weights, bias, text)

In [ ]:
# text = "that Samuel Lander founded the Williamston Female College in an abandoned hotel?"
# test(weights, bias, text)

In [ ]:
# text = "Operazione Quercia (in tedesco Unternehmen Eiche) fu il nome in codice dato all'operazione militare condotta il 12 settembre 1943 dai paracadutisti tedeschi della 2. Fallschirmjäger-Division che portò alla liberazione di Benito Mussolini dalla prigionia a Campo Imperatore, sul Gran Sasso. Dopo essere stato arrestato il 25 luglio 1943, Mussolini venne condotto in varie località e alla fine trasferito a Campo Imperatore a fine estate del 1943, una zona isolata e raggiungibile solo tramite funivia, dove era guardato a vista. Per non rischiare di farlo cadere in mano agli Alleati, Adolf Hitler ordinò al generale dei paracadutisti Kurt Student di organizzare una missione per la liberazione del Duce servendosi dei suoi Fallschirmjäger, a cui vennero aggregati, per ragioni politiche, sedici uomini del Servizio di sicurezza (Sicherheitsdienst - SD) delle SS agli ordini del capitano Otto Skorzeny."
# test(weights, bias, text)

In [ ]:
# text = "Il 12 settembre, pochi giorni dopo il Proclama Badoglio che annunciava la resa incondizionata delle forze italiane agli Alleati, i paracadutisti tedeschi lanciarono un audace assalto per liberare Mussolini, che si risolse con successo e senza perdite per gli assalitori. Grazie ai suoi contatti diretti con Ernst Kaltenbrunner e Heinrich Himmler, fin da subito Skorzeny riuscì a imporre la propria versione distorta e autocelebrativa dei fatti avvenuti sul Gran Sasso"
# test(weights, bias, text)

In [ ]:
for lr in [1, 0.1, 0.01, 0.001, 0.0001]:
    print(lr, 30 * "*")
    data, labels = load_data("./dataset/train")
    encoded_labels = encode_labels(labels)

    weights, bias = train(data, encoded_labels, epochs=15, learning_rate=lr)
    data, labels = load_data("./dataset/test")
    labels = encode_labels(labels)
    test(data, labels, weights, bias)

1 ******************************
Epoch 1/15 accuracy: 0.5
Epoch 2/15 accuracy: 0.6666666666666666
Epoch 3/15 accuracy: 1.0
Epoch 4/15 accuracy: 1.0
Epoch 5/15 accuracy: 1.0
Epoch 6/15 accuracy: 1.0
Epoch 7/15 accuracy: 1.0
Epoch 8/15 accuracy: 1.0
Epoch 9/15 accuracy: 1.0
Epoch 10/15 accuracy: 1.0
Epoch 11/15 accuracy: 1.0
Epoch 12/15 accuracy: 1.0
Epoch 13/15 accuracy: 1.0
Epoch 14/15 accuracy: 1.0
Epoch 15/15 accuracy: 1.0
Accuracy test: 0.7
0.1 ******************************
Epoch 1/15 accuracy: 0.16666666666666666
Epoch 2/15 accuracy: 0.16666666666666666
Epoch 3/15 accuracy: 0.0
Epoch 4/15 accuracy: 0.16666666666666666
Epoch 5/15 accuracy: 0.0
Epoch 6/15 accuracy: 0.16666666666666666
Epoch 7/15 accuracy: 0.0
Epoch 8/15 accuracy: 0.3333333333333333
Epoch 9/15 accuracy: 0.16666666666666666
Epoch 10/15 accuracy: 0.3333333333333333
Epoch 11/15 accuracy: 1.0
Epoch 12/15 accuracy: 1.0
Epoch 13/15 accuracy: 1.0
Epoch 14/15 accuracy: 1.0
Epoch 15/15 accuracy: 1.0
Accuracy test: 0.7
0.01 **